In [20]:
import pickle
import humanize
import os

def print_pickled_size(var_name, var):
    with open(f"./trash/{var_name}.pickle", 'wb') as f:
        pickle.dump(var, f)
    print(f'{var_name} requires {humanize.naturalsize(os.path.getsize(f"./trash/{var_name}.pickle"))}')

In [21]:
# tonno code

def VBEncode(n):
    byte = [] # We will store the bytes in a list
    while True:
        # n % 128 gives us the 7 least significant bits
        byte.append(n % 128) # Append 7 bits with most significant bit (control bit) set to 0
        if n < 128:
            break
        n //= 128 # Shift the number to the right by 7 bits

    byte[0] += 128 # Set the most significant bit (control bit) set to 1
    return byte[::-1] # We provide the resulting list in reverse order

def VBEncodeList(n_list):
    b = []
    for n in n_list:
        b.extend(VBEncode(n)) # We extend the list with the bytes of the number
    return b


def VBDecode(byte_list):
    n_list = []
    n = 0
    for b in byte_list:
        if b < 128:
            n = 128 * n + b
        else:
            n = 128 * n + b - 128
            n_list.append(n)
            n = 0
    return n_list

In [22]:
import random


def vbyte_encode(number):
    bytes_list = bytearray()
    while True:
        byte = number & 0x7F # Prendi i 7 bit meno significativi -> 0111 1111 = 0x7F
        number >>= 7 # Shifta a destra di 7 bit
        if number:
            bytes_list.append(byte) # Aggiungi i 7 bit al risultato
        else:
            bytes_list.append(0x80 | byte) # Aggiungi i 7 bit con il bit di continuazione, 0x80 = 1000 0000
            break
    return bytes(bytes_list)

def vbyte_decode(bytes_seq):
    number = 0
    for i, byte in enumerate(bytes_seq):
        number |= (byte & 0x7F) << (7 * i)
        if byte & 0x80:
            break
    return number

def decode_concatenated_vbyte(encoded_bytes):
    decoded_numbers = []
    current_number = 0
    shift_amount = 0
    
    for byte in encoded_bytes:
        if byte & 0x80:  # Bit di continuazione trovato, fine del numero
            current_number |= (byte & 0x7F) << shift_amount
            decoded_numbers.append(current_number)
            current_number = 0
            shift_amount = 0
        else:  # Continua a comporre il numero
            current_number |= (byte & 0x7F) << shift_amount
            shift_amount += 7
    
    return decoded_numbers



random.seed(42)
# Genera una lista di numeri casuali
numbers = [random.randint(1, 1000) for _ in range(1)]
print(f"Starting values: {numbers[:10]}...")  # Mostra solo i primi 10 per brevità
print_pickled_size('starting_values', numbers)

# Codifica ogni numero e crea un'unica bytearray
encoded_list = [vbyte_encode(x) for x in numbers]
print(encoded_list)
concatenated_encoded = b''.join(encoded_list)  # Concatenazione in un singolo bytes
print_pickled_size('encoded_list', concatenated_encoded)

# Decodifica
decoded_list = [vbyte_decode(x) for x in encoded_list]
print(f"Decoded list (first 10 values): {decoded_list[:10]}...")

decoded_list = decode_concatenated_vbyte(concatenated_encoded)
print(f"Decoded list (first 10 values): {decoded_list[:10]}...")



Starting values: [655]...
starting_values requires 18 Bytes
[b'\x0f\x85']
encoded_list requires 17 Bytes
Decoded list (first 10 values): [655]...
Decoded list (first 10 values): [655]...
